In [2]:
import numpy as np
import pandas as pd
import os

In [5]:
dataPath = os.path.join(os.getcwd(), 'adult_data.csv')
dataPath

'C:\\Users\\THIS PC\\Projects\\Learning\\Demographic Data Analyzer\\adult_data.csv'

In [6]:
df = pd.read_csv(dataPath)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# How many people of each race are represented in this dataset?

In [14]:
unq_races = pd.Series(df['race'].unique())
race_counts = pd.Series({
    r: (df['race'] == r).sum() for r in unq_races
})
race_counts

White                 27816
Black                  3124
Asian-Pac-Islander     1039
Amer-Indian-Eskimo      311
Other                   271
dtype: int64

# What is the average age of men?

In [29]:
men = df[['sex', 'age']][df['sex'] == 'Male']
men['age'].mean()

39.43354749885268

# What is the percentage of people who have a Bachelor's degree?

In [40]:
bac = df['education'][df['education'] == 'Bachelors'].count() / df['education'].count() * 100
print(f'{bac:.2f}%')

16.45%


# What percentage of people with advanced education (Bachelors, Masters, Doctorate) make more than 50K?

In [82]:
adv_edu = pd.Series(['Bachelors', 'Masters', 'Doctorate'])
adv_edu_50 = df[df['education'].isin(adv_edu)][['education', 'salary']]
adv_edu_50 = adv_edu_50[adv_edu_50['salary'] == '>50K']
print(f'{adv_edu_50['salary'].count() / df['salary'].count() * 100:.2f}%')

10.71%


# What percentage of people without advanced education make more than 50K?

In [92]:
non_adv_edu_50 = df[~df['education'].isin(adv_edu)][['education', 'salary']]
non_adv_edu_50 = non_adv_edu_50[non_adv_edu_50['salary'] == '>50K']
non_per = non_adv_edu_50['salary'].count() / df['salary'].count() * 100
print(f'{non_per:.2f}%')

13.37%


# What is the minimum number of hours a person works per week?

In [93]:
df['hours-per-week'].min()

1

# What percentage of the people who work the minimum number of hours per week have a salary of more than 50K?

In [105]:
min_50 = df[df['hours-per-week'] == df['hours-per-week'].min()][['hours-per-week', 'salary']]
per_min_50 = min_50[min_50['salary'] == '>50K']['hours-per-week'].count() / df.iloc[:, 0].count() * 100
print(f'{per_min_50:.2f}%')

0.01%


# What country has the highest percentage of people that earn >50K and what is that percentage?

In [153]:
country_counts = df['native-country'].value_counts()
country_rich_counts = df[df['salary'] == '>50K']['native-country'].value_counts()

country_rich_percentage = (country_rich_counts / country_counts * 100).round(1)

rich_df = pd.DataFrame([country_rich_percentage])
rich_df.index = ['% earning >50K']
rich_df.loc[:, rich_df.iloc[0].sort_values(ascending=False).index]

native-country,Iran,France,India,Taiwan,Japan,Yugoslavia,Cambodia,Italy,England,Canada,...,Honduras,Vietnam,Peru,Nicaragua,Mexico,Guatemala,Columbia,Dominican-Republic,Holand-Netherlands,Outlying-US(Guam-USVI-etc)
% earning >50K,41.9,41.4,40.0,39.2,38.7,37.5,36.8,34.2,33.3,32.2,...,7.7,7.5,6.5,5.9,5.1,4.7,3.4,2.9,NaN,NaN


# Identify the most popular occupation for those who earn >50K in India

In [185]:
india_rich = df[df['native-country'] == 'India'][['occupation', 'salary']]
india_rich = india_rich[india_rich['salary'] == '>50K']['occupation']
india_rich.to_frame().head()

,occupation
11,Prof-specialty
968,Exec-managerial
1327,Other-service
7258,Prof-specialty
7285,Prof-specialty


In [186]:
india_rich_occ = pd.Series({
    r: (india_rich.values == r).sum() for r in india_rich.unique()
}) #india_rich is Series -> use values instead of columns(DataFrame)
india_rich_occ

Prof-specialty      25
Exec-managerial      8
Other-service        2
Tech-support         2
Transport-moving     1
Sales                1
Adm-clerical         1
dtype: int32